# CMR Bulk

#### The Idea
By basically just adding an extra dimension to the arrays that CMR or InstanceCMR use to simulate encoding and retrieval, we can compute trial likelihoods across trials using just a single model instance. For example, currently CMR's context representation is a vector. We can add another dimension to that array with length T where T is the number of trials you're interested in performing simulations with. Same goes for representations like Mfc and Mcf. Where relevant during encoding (such as when item presentation scheme varies as in the Lohnas spacing dataset) or retrieval (recall order varies across trials), we can pass vectors of events instead of single events to track model performance using a single sequence of array operations. Before in my likelihood functions I reset or (worse) instantiated a totally new model instance for each trial I wanted to run my model on; with this approach, we can avoid that bottleneck completely while still having a codebase can can run single-trial simulations.

#### Testing Requirements
We need a version of CMR that can maintain and update representations of multiple model states simultaneously.
Our testing function should use this model to encode multiple sequences of experiences simultaneously, then to perform free recall each instance as well, using a single array operation across instances each time.

This still applies a for-loop: each new model state is generated sequentially.This is necessary as far as we can tell because succeeding states depend on prior states. But we may be able to move some operations outside that loop. 

If a record of relevant aspects of each model state is tracked across transitions within a cross-transition representation, for example, then computation of outcome_probabilities and similar operations can be performed across all states using a single set of operations as well. But we'll save that modification for later.

## Model

In [1]:
# export
# hide

import numpy as np
import math
from numba import float64, int32, boolean
from numba.experimental import jitclass

cmr_spec = [
    ('item_count', int32), 
    ('encoding_drift_rate', float64),
    ('start_drift_rate', float64),
    ('recall_drift_rate', float64),
    ('shared_support', float64),
    ('item_support', float64),
    ('learning_rate', float64),
    ('primacy_scale', float64),
    ('primacy_decay', float64),
    ('stop_probability_scale', float64),
    ('stop_probability_growth', float64),
    ('choice_sensitivity', float64),
    ('context', float64[:, ::1]),
    ('recall', int32[:, ::1]),
    ('retrieving', boolean),
    ('recall_total', int32),
    ('primacy_weighting', float64[::1]),
    ('probabilities', float64[:, ::1]),
    ('mfc', float64[:, :, ::1]),
    ('mcf', float64[:, :, ::1]),
    ('encoding_index', int32),
    ('items', float64[:,::1]),
    ('trial_count', int32),
    ('context_input', float64[:,::1])
]

In [2]:
# export

@jitclass(cmr_spec)
class CMR:

    def __init__(self, item_count, presentation_count, trial_count, 
                 encoding_drift_rate, start_drift_rate, recall_drift_rate, 
                 shared_support, item_support, learning_rate, primacy_scale, 
                 primacy_decay, stop_probability_scale, 
                 stop_probability_growth, choice_sensitivity):
        
        # store initial parameters
        self.item_count = item_count
        self.encoding_drift_rate = encoding_drift_rate
        self.start_drift_rate = start_drift_rate
        self.recall_drift_rate = recall_drift_rate
        self.shared_support = shared_support
        self.item_support = item_support
        self.learning_rate = learning_rate
        self.primacy_scale = primacy_scale
        self.primacy_decay = primacy_decay
        self.stop_probability_scale = stop_probability_scale
        self.stop_probability_growth = stop_probability_growth
        self.choice_sensitivity = choice_sensitivity
        
        # at the start of the list context is initialized with a state 
        # orthogonal to the pre-experimental context associated with the items
        self.context = np.zeros((trial_count, item_count+1))
        self.context[:, 0] = 1
        self.recall = np.zeros((trial_count, item_count), dtype='int32') # preallocation
        self.retrieving = False
        self.recall_total = 0
        
        # predefine primacy weighting vectors
        self.primacy_weighting = primacy_scale * np.exp(
            -primacy_decay * np.arange(presentation_count)) + 1

        # preallocate for outcome_probabilities
        self.probabilities = np.zeros((trial_count, item_count+1))
        
        # The two layers communicate with one another through two sets of 
        # associative connections represented by matrices Mfc and Mcf. Pre-
        # experimental Mfc is 1-learning_rate and pre-experimental Mcf is 
        # item_support for i=j. For i!=j, Mcf is shared_support.
        self.mfc = np.zeros((trial_count, item_count, item_count+1))
        self.mfc[:,] = np.eye(item_count, item_count+1, 1) * (1-learning_rate)
        self.mcf = np.zeros((trial_count, item_count+1, item_count))
        self.mcf[:,] = np.ones((item_count+1, item_count)) * shared_support
        for i in range(item_count):
            self.mcf[:, i+1, i] = item_support
        self.mcf[:,1,:] = 0
        self.encoding_index = 0
        self.items = np.eye(item_count, item_count)
        self.trial_count = trial_count
        self.context_input = np.zeros((trial_count, self.item_count+1))
        
    def experience(self, experiences):
        
        for i in range(len(experiences[0])):
            self.update_context(self.encoding_drift_rate, experiences[:, i])
            
            for j in range(self.trial_count):
                self.mfc[j] += self.learning_rate * np.outer(
                    self.context[j], experiences[j, i]).T
                self.mcf[j] += self.primacy_weighting[
                    self.encoding_index] * np.outer(
                    self.context[j], experiences[j, i])
                
            self.encoding_index += 1
            
    def update_context(self, drift_rate, experience=None):

        # first pre-experimental or initial context is retrieved
        self.context_input[:] = 0
        if experience is not None:
            for i in range(self.trial_count):
                self.context_input[i, :] = np.dot(experience[i], self.mfc[i])
                self.context_input[i, :] = self.context_input[i, :] / np.sqrt(
                    np.sum(np.square(self.context_input[i, :]))) # make len 1
        else:
            self.context_input[:, 0] = 1

        # updated context is sum of context and input, modulated by rho to 
        # have len 1 and some drift_rate
        rho = np.sqrt(1 + np.square(drift_rate) * (np.square(
            self.context * self.context_input) - 1)) - (drift_rate * (
            self.context * self.context_input))
        self.context = (rho * self.context) + (drift_rate * self.context_input)
        
    def activations(self, probe, use_mfc=False):
        
        if use_mfc:
            activation = np.zeros((self.trial_count, len(self.mfc[0, 0])))
            for i in range(self.trial_count):
                activation[i] = np.dot(probe[i], self.mfc[i]) + 10e-7
        else:
            activation = np.zeros((self.trial_count, len(self.mcf[0, 0])))
            for i in range(self.trial_count):
                activation[i] = np.dot(probe[i], self.mcf[i]) + 10e-7
        return activation
        
    def outcome_probabilities(self, activation_cue):

        activation = self.activations(activation_cue)
        activation = np.power(activation, self.choice_sensitivity)

        self.probabilities[:, 1:] = 0
        self.probabilities[:, 0] = min(self.stop_probability_scale * np.exp(
            self.recall_total * self.stop_probability_growth), 1.0  - (
            self.item_count * 10e-7))
        
        # also set stop probability to 1 where recall has terminated
        if self.recall_total > 0:
            self.probabilities[self.recall[:, self.recall_total-1] == 0, 0] = 1
        
        # track for each trial whether recall termination is guaranteed or not
        termination_not_guaranteed = self.probabilities[:, 0] < 1
        
        # suppress activation for already recalled items to 0
        for trial_index in range(self.trial_count):
            if termination_not_guaranteed[trial_index]:
                for each in self.recall[trial_index, :self.recall_total]:
                    activation[trial_index, each-1] = 0
                self.probabilities[trial_index, 1:] = (
                    1-self.probabilities[trial_index, 0]) * activation[trial_index] / np.sum(activation[trial_index])

        return self.probabilities
    
    def force_recall(self, choice=None):
        
        if not self.retrieving:
            self.update_context(self.start_drift_rate)
            self.retrieving = True

        if choice is None:
            pass
        else:
            self.recall[:, self.recall_total] = choice
            self.recall_total += 1
            self.update_context(self.recall_drift_rate, self.items[choice - 1])

        return self.recall[:, :self.recall_total]

## Test

In [10]:
import numpy as np
from numba import njit
from numba.typed import List
from numba.typed import List

#@njit(fastmath=True, nogil=True)
def cmr_murd_likelihood(
    data_to_fit, item_counts, encoding_drift_rate, start_drift_rate, 
    recall_drift_rate, shared_support, item_support, learning_rate, 
    primacy_scale, primacy_decay, stop_probability_scale, 
    stop_probability_growth, choice_sensitivity):
    
    result = 0.0
    for i in range(len(item_counts)):
        item_count = item_counts[i]
        trials = data_to_fit[i]
        
        model = CMR(item_count, item_count, len(trials), encoding_drift_rate, 
                    start_drift_rate, recall_drift_rate, shared_support,
                    item_support, learning_rate, primacy_scale, 
                    primacy_decay, stop_probability_scale, 
                    stop_probability_growth, choice_sensitivity)

        # same sequence of experiences across trials
        experiences = np.zeros((len(trials), item_count, item_count))
        experiences[:,] = np.eye(item_count, item_count)
        model.experience(experiences)

        likelihood = np.ones((len(trials), item_count), dtype='float64')

        model.force_recall()
        for recall_index in range(len(trials[0]) + 1):

            # identify index of item recalled; if zero then recall is over
            if recall_index == len(trials[0]) and len(trials[0]) < item_count:
                recall = np.zeros(len(trials), dtype='int64')
            else:
                recall = trials[:, recall_index]

            # store probability of and simulate recalling item with this index
            probs = model.outcome_probabilities(model.context)
            for j in range(len(trials)):
                likelihood[j, recall_index] = probs[j, recall]

            if np.all(recall == 0):
                break
            model.force_recall(recall)
        
        result -= np.sum(np.log(likelihood))

    return result

In [11]:
from instance_cmr.datasets import *

murd_trials0, murd_events0, murd_length0 = prepare_murddata(
    '../data/MurdData_clean.mat', 0)
print(murd_length0, np.shape(murd_trials0))

murd_events0.head()

20 (1200, 15)


,subject,list,item,input,output,study,recall,repeat,intrusion
0,1,1,1,1,5.0,True,True,0,False
1,1,1,2,2,7.0,True,True,0,False
2,1,1,3,3,NaN,True,False,0,False
3,1,1,4,4,NaN,True,False,0,False
4,1,1,5,5,NaN,True,False,0,False


In [12]:
lb = np.finfo(float).eps
hand_fit_parameters = {
    'item_counts': List([murd_length0]),
    'encoding_drift_rate': .8,
    'start_drift_rate': .7,
    'recall_drift_rate': .8,
    'shared_support': 0.01,
    'item_support': 1.0,
    'learning_rate': .3,
    'primacy_scale': 1,
    'primacy_decay': 1,
    'stop_probability_scale': 0.01,
    'stop_probability_growth': 0.3,
    'choice_sensitivity': 2
}
cmr_murd_likelihood(List([murd_trials0[:80]]), **hand_fit_parameters)

ValueError: setting an array element with a sequence.

In [9]:
%%timeit
cmr_murd_likelihood(List([murd_trials0[:80]]), **hand_fit_parameters)

ValueError: setting an array element with a sequence.

## Dot Product Explorations

In [386]:
a = np.arange(48).reshape(8, 6)
a

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41],
       [42, 43, 44, 45, 46, 47]])

In [393]:
np.outer(a[0], a[0])

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5],
       [ 0,  2,  4,  6,  8, 10],
       [ 0,  3,  6,  9, 12, 15],
       [ 0,  4,  8, 12, 16, 20],
       [ 0,  5, 10, 15, 20, 25]])

In [394]:
print(np.shape(np.outer(a, a)))
np.outer(a, a)[0:6,0:6]

(48, 48)


array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5],
       [ 0,  2,  4,  6,  8, 10],
       [ 0,  3,  6,  9, 12, 15],
       [ 0,  4,  8, 12, 16, 20],
       [ 0,  5, 10, 15, 20, 25]])

In [407]:
np.shape(np.outer(a, a))

(48, 48)

In [362]:
b = np.arange(336).reshape((8, 6, 7))
b[0]

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27],
       [28, 29, 30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39, 40, 41]])

In [313]:
for i in range(8):
    print(a[i] @ b[i])

[0 1 2 3 4 5 6]
[49 50 51 52 53 54 55]
[ 98  99 100 101 102 103 104]
[147 148 149 150 151 152 153]
[196 197 198 199 200 201 202]
[245 246 247 248 249 250 251]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]


In [314]:
print(np.shape(np.dot(a, b)))
result = np.dot(a, b)

result[np.eye(8, dtype='bool')]

(8, 8, 7)


array([[  0,   1,   2,   3,   4,   5,   6],
       [ 49,  50,  51,  52,  53,  54,  55],
       [ 98,  99, 100, 101, 102, 103, 104],
       [147, 148, 149, 150, 151, 152, 153],
       [196, 197, 198, 199, 200, 201, 202],
       [245, 246, 247, 248, 249, 250, 251],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0]])

In [307]:
np.eye(8, dtype='bool')[0] +1

array([2, 1, 1, 1, 1, 1, 1, 1])

In [346]:
np.arange(48).reshape(6, 8) * np.arange(48).reshape(6, 8)

array([[   0,    1,    4,    9,   16,   25,   36,   49],
       [  64,   81,  100,  121,  144,  169,  196,  225],
       [ 256,  289,  324,  361,  400,  441,  484,  529],
       [ 576,  625,  676,  729,  784,  841,  900,  961],
       [1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521],
       [1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209]])

In [347]:
np.arange(48).reshape(6, 8)[0] ** 2

array([ 0,  1,  4,  9, 16, 25, 36, 49], dtype=int32)

## Development Notes

During initialization, the main change is just setting `trial_count` as the first dimension of whatever was previously a vector or 2D array.

`self.primacy_weighting` seems to be an exception; this changes if we decide the model should be able to accept vectors of parameter values to vary across trials. 

In [50]:
trial_count = 100
item_count = 20
learning_rate = .8
shared_support = .02
item_support = .7

context = np.zeros((trial_count, item_count + 1))

In [19]:
context[:, 0] = 1

In [20]:
context

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

As for Mfc and Mcf, `np.eye` doesn't support higher than two dimensions. I have limited options for repeating the operation over a dimension, too, thanks to numba limitations. Maybe the approach is to stay with a zero array with the right size, and assign the eye array across trials simultaneously.

In [26]:
mfc = np.zeros((trial_count, item_count, item_count + 1))

In [30]:
mfc[:,] = np.eye(item_count, item_count+1, 1) * (1 - learning_rate)
np.all(mfc[1] == np.eye(item_count, item_count+1, 1) * (1 - learning_rate))

True

In [40]:
mcf = np.zeros((trial_count, item_count, item_count))
mcf[:, ] = np.ones((item_count, item_count)) * shared_support

In [43]:
%%timeit
for i in range(item_count):
    mcf[:, i, i] = shared_support

9.38 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [46]:
%%timeit
ref = np.eye(item_count) * shared_support
mcf[:, ] = ref

9.63 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [48]:
np.vstack((np.array([1, 2]), np.array([1, 3])))

array([[1, 2],
       [1, 3]])

In [ ]:
mfc = np.zeros((trial_count, item_count, item_count + 1))
mfc[:,] = np.eye(item_count, item_count+1, 1) * (1 - learning_rate)
mcf = np.zeros((trial_count, item_count+1, item_count))
mcf[:,] = np.ones((item_count+1, item_count)) * shared_support
for i in range(item_count):
    mcf[:, i+1, i] = item_support
mcf[:,0,:] = 0

mcf[0]

In [56]:
parameters = {
    'item_count': 20,
    'presentation_count': 20,
    'trial_count': 100,
    'encoding_drift_rate': .8,
    'start_drift_rate': .7,
    'recall_drift_rate': .8,
    'shared_support': 0.01,
    'item_support': 1.0,
    'learning_rate': .3,
    'primacy_scale': 1,
    'primacy_decay': 1,
    'stop_probability_scale': 0.01,
    'stop_probability_growth': 0.3,
    'choice_sensitivity': 2
}

model = CMR(**parameters)

In [59]:
model.mfc[0]

array([[0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0